<a href="https://colab.research.google.com/github/KAMBLE/Model-Deployment/blob/main/Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Deploy Model using Flask and Colab**



---









In [ ]:
!pip install transformers

In [3]:
from transformers import (TFBertForSequenceClassification, 
                          BertTokenizer)
#from transformers import BertConfig,BertModel
import tensorflow as tf
#import numpy as np

In [9]:
import glob
from google.colab import drive
drive.mount('/content/bert')

Mounted at /content/bert


In [10]:
!ls

bert  sample_data  senti_class


In [ ]:
#config_file = glob.glob(f"/content/drive/config.json")
#model_file = glob.glob(f"/content/drive/tf_model.h5")
#model_path = glob.glob(f"/content/drive/senti_class/")

In [11]:
load_model = TFBertForSequenceClassification.from_pretrained('/content/bert/My Drive/senti_class')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/bert/My Drive/senti_class.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [13]:
load_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# sample text prediction on pretrain loaded model
review_text = 'NKD down day in lat going to wear off the overbought while giving back nothing amazing	'
inputs = tokenizer(review_text, return_tensors="tf")
argmx_res = tf.nn.softmax(load_model(inputs))
pred_class = tf.math.argmax(argmx_res[0][0],axis=0)
#pred_class_list.append(np.array([pred_class])[0])

print('predicted sentiment : ',np.array([pred_class])[0])

predicted sentiment :  1


## Test Flask ....

In [14]:
!pip install flask-ngrok

In [16]:
from flask_ngrok import run_with_ngrok
from flask import Flask,send_from_directory,request,render_template
app = Flask(__name__,template_folder='/content/bert/My Drive/senti_class/')
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    ip_text =  request.values.get("comment") 
    inputs = tokenizer(ip_text, return_tensors="tf")
    argmx_res = tf.nn.softmax(load_model(inputs))
    pred_class = tf.math.argmax(argmx_res[0][0],axis=0)

    return render_template('index.html', prediction_text='Sentiment is  {}'.format(pred_class))
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5bc6211ed8a4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Oct/2020 05:33:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 05:33:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2020 05:33:40] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 05:33:48] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 05:43:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 05:43:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2020 05:44:08] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 07:19:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 07:19:37] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 07:21:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 07:23:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 07:23:15] "GET / HTTP/1.1" 200 -
